In [1]:
import cv2
import numpy as np

In [2]:
# Dictionary of color ranges
color_ranges = {
    'white': ([0, 0, 200], [180, 30, 255]),
    'gray': ([0, 0, 100], [180, 50, 199]),
    'black': ([0, 0, 0], [180, 255, 50]),
    'blue': ([90, 50, 50], [150, 255, 255]),
    'red': ([0, 50, 50], [10, 255, 255]),
    'green': ([40, 50, 50], [80, 255, 255]),
    'magenta': ([140, 50, 50], [170, 255, 255]),
    'yellow': ([20, 50, 50], [40, 255, 255]),
    'turquoise': ([80, 50, 50], [100, 255, 255])
}

In [3]:
def detect_color(frame):
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    for color, (lower, upper) in color_ranges.items():
        lower = np.array(lower, dtype=np.uint8)
        upper = np.array(upper, dtype=np.uint8)
        
        mask = cv2.inRange(hsv_frame, lower, upper)
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            moments = cv2.moments(largest_contour)
            
            # m00 is total Area of the countor
            # m10 is sum of all x-coordinates of contour points
            # m01 is sum of all y-coordinates of contour points
            if moments["m00"] != 0:
                cx = int(moments["m10"] / moments["m00"]) 
                cy = int(moments["m01"] / moments["m00"])
                cv2.putText(frame, color, (cx, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    return frame

In [4]:
cap = cv2.VideoCapture('input/color.mp4')

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output/color_detection_output.mp4', fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

In [5]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    frame = detect_color(frame)
    # Write the frame to the output video
    out.write(frame)
    
    cv2.imshow('Color Detection', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()